In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Initialize the WebDriver
chromedriver_path = 'chromedriver.exe'  # Path to your chromedriver
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service)

# URL of the page to scrape
url = 'https://www.hondacengkareng.com/produk-diskon/'
driver.get(url)
driver.implicitly_wait(10)

# Lists to store product details
nama_produk = []
kategori = []
harga_sebelumnya = []
harga_sekarang = []

def click_all_product_links_on_page():
    try:
        # Tunggu elemen produk muncul
        links = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'woocommerce-loop-product__title'))
        )
    except Exception as e:
        print(f"Error occurred while finding product links: {e}")
        return

    # Cek apakah ada link yang ditemukan
    if not links:
        print("Tidak ada produk yang ditemukan di halaman ini.")
        return

    for i in range(len(links)):
        try:
            # Ambil elemen kembali setiap iterasi untuk memastikan DOM terbaru
            links = driver.find_elements(By.CLASS_NAME, 'woocommerce-loop-product__title')
            if i >= len(links):
                print(f"Indeks {i} tidak ditemukan dalam daftar links.")
                continue

            link = links[i]
            link.click()

            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'product_title.entry-title'))
            )

            # Mengambil informasi produk
            try:
                title = driver.find_element(By.CLASS_NAME, 'product_title.entry-title')
                product_name = title.text
                nama_produk.append(product_name)
            except Exception as e:
                print(f"Error occurred while retrieving product name: {e}")
                nama_produk.append('')

            try:
                # Mengambil harga sebelumnya dari elemen <del>
                del_amount = driver.find_element(By.CSS_SELECTOR, "del .woocommerce-Price-amount")
                harga_sebelumnya_value = del_amount.text.replace('Rp', '').replace(',', '').strip()
                harga_sebelumnya.append(float(harga_sebelumnya_value))

                # Mengambil harga sekarang dari elemen <ins>
                ins_amount = driver.find_element(By.CSS_SELECTOR, "ins .woocommerce-Price-amount")
                harga_sekarang_value = ins_amount.text.replace('Rp', '').replace(',', '').strip()
                harga_sekarang.append(float(harga_sekarang_value))
            except Exception as e:
                print(f"Error occurred while retrieving product price: {e}")
                harga_sebelumnya.append(0)
                harga_sekarang.append(0)

            try:
                # Mengambil kategori produk
                category = driver.find_element(By.CSS_SELECTOR, 'span.posted_in')
                category_text = category.text.replace('Kategori: ', '').strip()
                if category_text:  # Only append non-empty categories
                    kategori.append(category_text)
                else:
                    kategori.append('Tidak ada kategori')  # If empty, append a placeholder
            except Exception as e:
                print(f"Error occurred while retrieving product category: {e}")
                kategori.append('Tidak ada kategori')

            # Cetak detail produk untuk verifikasi
            print("Nama produk:", nama_produk[-1])
            print("Harga sebelumnya:", harga_sebelumnya[-1])
            print("Harga sekarang:", harga_sekarang[-1])
            print("Kategori produk:", kategori[-1])
            
            # Kembali ke halaman sebelumnya
            driver.back()
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'woocommerce-loop-product__title'))
            )
            
        except Exception as e:
            print(f"Error occurred while clicking or retrieving product information: {e}")

# Ulangi untuk setiap halaman
total_pages = 1  # Ubah ke jumlah total halaman
for page_num in range(1, total_pages + 1):
    print(f"Processing page {page_num}")
    
    click_all_product_links_on_page()
    
    # Navigasi ke halaman berikutnya jika ada
    if page_num < total_pages:
        try:
            next_page_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.LINK_TEXT, str(page_num + 1)))
            )
            next_page_button.click()
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'woocommerce-loop-product__title'))
            )
        except Exception as e:
            print(f"Could not navigate to the next page: {e}")

# Cetak semua detail produk yang dikumpulkan di akhir
print("All collected product names:", nama_produk)
print("All collected previous prices:", harga_sebelumnya)
print("All collected current prices:", harga_sekarang)

# Hanya cetak kategori yang tidak kosong
print("All collected categories:", [k for k in kategori if k != 'Tidak ada kategori'])



Processing page 1
Nama produk: AHM Oil SPX2 – 0.8 L
Harga sebelumnya: 69500.0
Harga sekarang: 66025.0
Kategori produk: Oli
Nama produk: Oli Gardan Transmission Gear Oil (Fully Synthetic)
Harga sebelumnya: 19500.0
Harga sekarang: 18525.0
Kategori produk: Oli
Nama produk: Oli Gardan – Transmission Gear Oil
Harga sebelumnya: 16500.0
Harga sekarang: 15675.0
Kategori produk: Oli
Nama produk: AHM Oil MPX2 – 0.8 L 10W-30
Harga sebelumnya: 58500.0
Harga sekarang: 55575.0
Kategori produk: Oli
Nama produk: AHM Oil SPX2 – 0.65 L 10W-30
Harga sebelumnya: 60500.0
Harga sekarang: 57475.0
Kategori produk: Oli
Nama produk: AHM Oil MPX2 – 0.65 L 10W-30
Harga sebelumnya: 49500.0
Harga sekarang: 47025.0
Kategori produk: Oli
Nama produk: Fender Eliminator Resmi New Honda CB150R StreetFire
Harga sebelumnya: 384000.0
Harga sekarang: 288000.0
Kategori produk: Aksesoris Resmi Motor Honda
Nama produk: AHM Oil MPX1 – 0.8 L 10W-30
Harga sebelumnya: 56500.0
Harga sekarang: 53675.0
Kategori produk: Oli
Nama produk

In [3]:
import pandas as pd
df_nama_produk = pd.DataFrame(nama_produk, columns=["Nama Produk"])
df_harga_sebelumnya = pd.DataFrame(harga_sebelumnya, columns=["Harga"])
df_harga_sekarang = pd.DataFrame(harga_sekarang, columns=["Kode Produk"])
df_kategori = pd.DataFrame(kategori, columns=["Kategori"])




# Combine all DataFrames into a single DataFrame
final_df = pd.concat([
    df_nama_produk,
    df_harga_sebelumnya,
    df_harga_sekarang,
    df_kategori,
   
], axis=1)

print(final_df)


# Save the DataFrame to a CSV file
final_df.to_csv('oli.csv', index=False, sep=';', encoding='utf-8')

print("DataFrame has been saved to 'produk_diskon.csv'")



                                          Nama Produk     Harga  Kode Produk  \
0                                AHM Oil SPX2 – 0.8 L   69500.0      66025.0   
1   Oli Gardan Transmission Gear Oil (Fully Synthe...   19500.0      18525.0   
2                  Oli Gardan – Transmission Gear Oil   16500.0      15675.0   
3                         AHM Oil MPX2 – 0.8 L 10W-30   58500.0      55575.0   
4                        AHM Oil SPX2 – 0.65 L 10W-30   60500.0      57475.0   
5                        AHM Oil MPX2 – 0.65 L 10W-30   49500.0      47025.0   
6   Fender Eliminator Resmi New Honda CB150R Stree...  384000.0     288000.0   
7                         AHM Oil MPX1 – 0.8 L 10W-30   56500.0      53675.0   
8                         AHM Oil SPX1 – 0.8 L 10W-30   68500.0      65075.0   
9        Paket Aksesoris All New Scoopy Stylish Brown  355500.0     175000.0   
10                  Garnish Headlight Honda Vario 160  214000.0     149800.0   
11       Paket Aksesoris All New Scoopy 

In [4]:
import csv
from IPython.display import FileLink, display  # Import FileLink and display
# Path to save the CSV file
file_path = 'produk_diskon.csv'

# Data headers
headers = ['Nama Produk', 'Harga_Sebelumnya', 'Harga_Sekarang', 'Kategori', 'Fitur']

# Data rows
rows = zip(nama_produk, harga_sebelumnya, harga_sekarang, kategori)

# Write the CSV file with UTF-8 BOM encoding
with open(file_path, mode='w', encoding='utf-8-sig', newline='') as file:
    writer = csv.writer(file, delimiter=';')
    
    # Write the headers
    writer.writerow(headers)
    
    # Write the rows
    writer.writerows(rows)

print(f"Data successfully written to {file_path}")

# Generate a download link
display(FileLink(file_path))

Data successfully written to produk_diskon.csv


c:\Users\basti\OneDrive\Documents\ds_salary_proj\produk_diskon.csv